In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from spacy.lang.fr import French
parser = French()
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
#from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import spacy
nlp=spacy.load("fr_core_news_md")
import textacy

In [2]:
X_full = pd.read_csv('allDataWithLabel.csv',encoding="utf-8")
X_test_full = pd.read_csv('allDataWithLabel.csv',encoding="utf-8")

y=X_full[["label"]]
X_full=X_full.drop(["label"],axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.5, test_size=0.5,
                                                                random_state=1)

In [3]:
spacy_stopwords = spacy.lang.fr.stop_words.STOP_WORDS
stop=list(spacy_stopwords)


In [4]:
def spacy_tokenizer(sentence):
    sentence=textacy.preprocess_text(sentence,lowercase=True,no_punct=True,no_urls=True
                                    ,no_emails=True,no_currency_symbols=True)
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word.strip().lower() for word in mytokens if word.strip().lower() not in stop ]
    return mytokens

In [5]:
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)
#classifier = SVC(C=150, gamma=2e-2, probability=True)
from sklearn.ensemble import RandomForestClassifier

classifier = sklearn.ensemble.RandomForestClassifier() 
pipe=Pipeline([("vectorizer",tfvectorizer),("classifier",classifier)])

In [6]:
from sklearn.base import TransformerMixin 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}
def clean_text(text):     
    return text.strip().lower()

In [7]:
from sklearn.preprocessing import StandardScaler

pipe.fit(X_train["article"],y_train)
pred=pipe.predict(X_valid["article"])


/home/mo5/Desktop/desktop/python/jupyter1.0/jupyter1.0_env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/mo5/Desktop/desktop/python/jupyter1.0/jupyter1.0_env/lib/python3.7/site-packages/sklearn/pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


In [8]:
accuracy_score(y_valid["label"],pred)

0.9829059829059829

In [13]:
pred

array(['autres', 'nahdha', 'nahdha', ..., 'autres', 'nahdha', 'autres'],
      dtype=object)

In [9]:
for i,res in zip(y_valid["label"],pred):
    #if(res==i): print("true")
    #else : print("false")
    print("{} : {}".format(i,res))

autres : autres
nahdha : nahdha
nahdha : nahdha
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
nidaa : nidaa
autres : autres
autres : autres
autres : autres
nidaa : nidaa
nidaa : autres
autres : autres
autres : autres
nahdha : nahdha
autres : autres
autres : autres
nidaa : nidaa
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
nidaa : nidaa
autres : autres
nidaa : nidaa
autres : autres
autres : autres
nahdha : nahdha
autres : autres
nidaa : nidaa
nidaa : nidaa
autres : autres
autres : autres
autres : autres
autres : autres
nidaa : nidaa
autres : autres
autres : autres
autres : autres
nidaa : nidaa
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
autres : autres
nidaa : nidaa
nidaa : nidaa
autres : autres
nahdha : nahdha
autres : autres
nahdha : nahdha
autres : autres

In [18]:
def accScore(sentence,res):
    
    predtest=pipe.predict([sentence])
    return predtest
    #return accuracy_score(res,predtest)

    

In [21]:
accScore(sentence="béji caied sebsi",res="nidaa")

array(['autres'], dtype=object)

preds_test = pipe.predict(X_test_full.values.reshape(-1,))
output = pd.DataFrame({'id': X_test_full.index,
                       'label': preds_test})
output.to_csv('test_predictions.csv', index=False)